<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/StabilityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

import sympy
from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig
from scipy.integrate import solve_ivp
from scipy.optimize import root, minimize

In [23]:
Ac = 0.9
b = 0.29
Kc=1.
qin_ss = 1.12
h_initial_ss = np.full(3, qin_ss/b)
h3sp_initial = h_initial_ss[-1]


def rhs(t, v, h3sp):
    h1, h2, h3 = v
    err = h3sp - h3
    q12 = b*h1
    q23 = b*h2
    q3 = b*h3
    qin1 = qin_ss + Kc*err

    dh1  = (qin1 - q12)/Ac
    dh2 = (q12 - q23)/Ac
    dh3 = (q23 - q3 )/Ac
    return jnp.array([dh1, dh2, dh3])

In [24]:
h3sp_initial

3.862068965517242

In [25]:
tend=100
res=solve_ivp(lambda t, v: rhs(t, v, h3sp_initial+1.), (0,tend ), h_initial_ss, method='Radau', dense_output=True)
res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  8.809e-02 ...  8.760e+01  1.000e+02]
        y: [[ 3.862e+00  3.959e+00 ...  4.636e+00  4.638e+00]
            [ 3.862e+00  3.863e+00 ...  4.637e+00  4.638e+00]
            [ 3.862e+00  3.862e+00 ...  4.638e+00  4.637e+00]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7c7d242e34c0>
 t_events: None
 y_events: None
     nfev: 170
     njev: 1
      nlu: 24

In [26]:
tplot = np.linspace(0,tend,100)
fig=make_subplots()
# fig.add_scatter(x=t,y=h3_dev+h_initial_ss[2], name='linear')
fig.add_scatter(x=tplot, y= res.sol(tplot)[2], name='solve_ivp')
fig.update_layout(width=600, height=400, template='plotly_dark')

In [27]:
K=1/b
Kc*K/(1+Kc*K)

0.7751937984496123

In [29]:
res.sol(tplot)[2][-1] - res.sol(tplot)[2][0]

0.775365621016415